# Acquire the Data

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from env import api_key

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

For this project, we will be using data in a timeline format. Ultimately, our model will be using a snapshot of the match at the 20 minute mark.

In [2]:
#Grab timeline data
response = requests.get(f'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4083906028/timeline/?api_key={api_key}')

In [3]:
#Turn it into json format
data = response.json()

In [4]:
data.keys()

dict_keys(['metadata', 'info'])

In [5]:
data['metadata']

{'dataVersion': '2',
 'matchId': 'NA1_4083906028',
 'participants': ['l5armf8S4PORoTNqEb9UcVkEvDzSYClySmYY4yQFBUU2ciCjRfQJLP9x-6uIMjmeUSm7x8-JnEWQxg',
  'idOuCKMdqN2RMvthmMVRFT8FXZSVnkmji_qWCXptdRpdltnNyXkEgNEj5Kl52iiJAVmGh6TFsZafdg',
  '5LQq6DKdoh-5hRyVrR6BdhviqccyYO7I-iK_s488B3Y8RtNdiOBlBQUkd-hw-5CzHh_LJU1eoQp4Tg',
  'p5XGEgLDT77MrwEYqvKAnWcezJHzn7BvrldHu9LbYJw5yVyaVPblF8MDQXQMPglMBESL7uTk9XWhsA',
  'YuzBuj8BE28q8sCt-H4Zjmh1Z3ASgXN3Zr4vgt2zmZgdO-MkFoIBrOWIX1qvtmXEfWiVV5tLmeB84w',
  'jBFJb2VptK-A5krGaWXc2srhMnkx7vHgco8vs2vabkaXKQ9wPTLzxZdM7TH2CQUkBYyum17N-vsobQ',
  'YMbwqbdNhRFlAXpnE38kPHt5qdrqD5_clK0aJR-dsFCH7hElbD4H6XIRa6GmKYB63tT_BXt2STIFuA',
  'oRiPFVFS8450BfEkJ8x4DtsFHPdVbBiHJsbhC1qZ5I1cYMuPB4zXqF9rwqG6R2w5LX0n7dd2b2OeGw',
  'eYxF_0xQzhoxKZG4uIGOariy2Cr1arjGRDosg_7-PLtJ2QwrPVq3y_T6LG8UXfMN-yPRhwhdhD2bBg',
  'F1cPo_wDQAeppIrKnDD981FiFo9tYm5Ttkp3gQ8K4ydvHoC-ChLkCmEHtM5TBkdLXzVdFJU5HbkweA']}

In [6]:
#Look at the data at the beginning of the match
data['info']['frames'][0]

{'events': [{'realTimestamp': 1635396104102,
   'timestamp': 0,
   'type': 'PAUSE_END'}],
 'participantFrames': {'1': {'championStats': {'abilityHaste': 0,
    'abilityPower': 0,
    'armor': 29,
    'armorPen': 0,
    'armorPenPercent': 0,
    'attackDamage': 25,
    'attackSpeed': 100,
    'bonusArmorPenPercent': 0,
    'bonusMagicPenPercent': 0,
    'ccReduction': 0,
    'cooldownReduction': 0,
    'health': 541,
    'healthMax': 541,
    'healthRegen': 0,
    'lifesteal': 0,
    'magicPen': 0,
    'magicPenPercent': 0,
    'magicResist': 30,
    'movementSpeed': 335,
    'omnivamp': 0,
    'physicalVamp': 0,
    'power': 200,
    'powerMax': 200,
    'powerRegen': 0,
    'spellVamp': 0},
   'currentGold': 500,
   'damageStats': {'magicDamageDone': 0,
    'magicDamageDoneToChampions': 0,
    'magicDamageTaken': 0,
    'physicalDamageDone': 0,
    'physicalDamageDoneToChampions': 0,
    'physicalDamageTaken': 0,
    'totalDamageDone': 0,
    'totalDamageDoneToChampions': 0,
    'tota

There are 2 sections to each index and each index appears to represent one minute of gameplay. The only exception is index 0, which is just a snapshot of the very beginning. 

The first section is 'events'. This section lists all of the events that occurred within that timeframe. Events are things like player kills, player deaths, level ups, etc. Joshua Chavez is working on turning all of the data in this section into a usable format.

The second section is 'particpantFrames'. This section is a dictionary of all players and their stats at the end of that timeframe. Stats are things like damage done, current gold, current level, total gold, etc. I will be working on this section and get all of the data in a usable format.

In [7]:
#Select the particpantFrames section for the very first timeframe
players = data['info']['frames'][0]['participantFrames']
players

{'1': {'championStats': {'abilityHaste': 0,
   'abilityPower': 0,
   'armor': 29,
   'armorPen': 0,
   'armorPenPercent': 0,
   'attackDamage': 25,
   'attackSpeed': 100,
   'bonusArmorPenPercent': 0,
   'bonusMagicPenPercent': 0,
   'ccReduction': 0,
   'cooldownReduction': 0,
   'health': 541,
   'healthMax': 541,
   'healthRegen': 0,
   'lifesteal': 0,
   'magicPen': 0,
   'magicPenPercent': 0,
   'magicResist': 30,
   'movementSpeed': 335,
   'omnivamp': 0,
   'physicalVamp': 0,
   'power': 200,
   'powerMax': 200,
   'powerRegen': 0,
   'spellVamp': 0},
  'currentGold': 500,
  'damageStats': {'magicDamageDone': 0,
   'magicDamageDoneToChampions': 0,
   'magicDamageTaken': 0,
   'physicalDamageDone': 0,
   'physicalDamageDoneToChampions': 0,
   'physicalDamageTaken': 0,
   'totalDamageDone': 0,
   'totalDamageDoneToChampions': 0,
   'totalDamageTaken': 0,
   'trueDamageDone': 0,
   'trueDamageDoneToChampions': 0,
   'trueDamageTaken': 0},
  'goldPerSecond': 0,
  'jungleMinionsKille

In [8]:
players['1']['currentGold']

500

In [9]:
#Now that I have the participantFrames, gather all the data and put it into a dict
#Will need to loop through each player.

players_dict = {}

for i in range(1, len(players) + 1):
    #Now create a temp dict to store the current players stats
    #Use formatted strings to automatically increment the player label
    temp_dict = {
        f'currentGold_{i}': players[str(i)]['currentGold'],
        f'magicDamageDoneToChampions_{i}': players[str(i)]['damageStats']['magicDamageDoneToChampions'],
        f'physicalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['physicalDamageDoneToChampions'],
        f'trueDamageDoneToChampions_{i}': players[str(i)]['damageStats']['trueDamageDoneToChampions'],
        f'totalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['totalDamageDoneToChampions'],
        f'goldPerSecond_{i}': players[str(i)]['goldPerSecond'],
        f'jungleMinionsKilled_{i}': players[str(i)]['jungleMinionsKilled'],
        f'level_{i}': players[str(i)]['level'],
        f'minionsKilled_{i}': players[str(i)]['minionsKilled'],
        f'timeEnemySpentControlled_{i}': players[str(i)]['timeEnemySpentControlled'],
        f'totalGold_{i}': players[str(i)]['totalGold'],
        f'xp_{i}': players[str(i)]['xp']
    }
    
    #Now that I have the current players stats, extend it to the overall players_dict
    players_dict.update(temp_dict)
    

In [10]:
players_dict

{'currentGold_1': 500,
 'magicDamageDoneToChampions_1': 0,
 'physicalDamageDoneToChampions_1': 0,
 'trueDamageDoneToChampions_1': 0,
 'totalDamageDoneToChampions_1': 0,
 'goldPerSecond_1': 0,
 'jungleMinionsKilled_1': 0,
 'level_1': 1,
 'minionsKilled_1': 0,
 'timeEnemySpentControlled_1': 0,
 'totalGold_1': 500,
 'xp_1': 0,
 'currentGold_2': 500,
 'magicDamageDoneToChampions_2': 0,
 'physicalDamageDoneToChampions_2': 0,
 'trueDamageDoneToChampions_2': 0,
 'totalDamageDoneToChampions_2': 0,
 'goldPerSecond_2': 0,
 'jungleMinionsKilled_2': 0,
 'level_2': 1,
 'minionsKilled_2': 0,
 'timeEnemySpentControlled_2': 0,
 'totalGold_2': 500,
 'xp_2': 0,
 'currentGold_3': 500,
 'magicDamageDoneToChampions_3': 0,
 'physicalDamageDoneToChampions_3': 0,
 'trueDamageDoneToChampions_3': 0,
 'totalDamageDoneToChampions_3': 0,
 'goldPerSecond_3': 0,
 'jungleMinionsKilled_3': 0,
 'level_3': 1,
 'minionsKilled_3': 0,
 'timeEnemySpentControlled_3': 0,
 'totalGold_3': 500,
 'xp_3': 0,
 'currentGold_4': 500,

In [11]:
#Now that I have been able to get the player stats in one time frame,
#Create a loop to get the rest

#I will need a list to store all the dictionaries from each timeframe
player_stats = []

#Here, each timeframe represents about one minute
for timeframe in data['info']['frames']:
    players = timeframe['participantFrames']
    
    #Now create a dicitonary to hold the players' stats from this timeframe
    players_dict = {}
    
    for i in range(1, len(players) + 1):
        #Now create a temp dict to store the current players stats
        #Use formatted strings to automatically increment the player label
        temp_dict = {
            f'currentGold_{i}': players[str(i)]['currentGold'],
            f'magicDamageDoneToChampions_{i}': players[str(i)]['damageStats']['magicDamageDoneToChampions'],
            f'physicalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['physicalDamageDoneToChampions'],
            f'trueDamageDoneToChampions_{i}': players[str(i)]['damageStats']['trueDamageDoneToChampions'],
            f'totalDamageDoneToChampions_{i}': players[str(i)]['damageStats']['totalDamageDoneToChampions'],
            f'goldPerSecond_{i}': players[str(i)]['goldPerSecond'],
            f'jungleMinionsKilled_{i}': players[str(i)]['jungleMinionsKilled'],
            f'level_{i}': players[str(i)]['level'],
            f'minionsKilled_{i}': players[str(i)]['minionsKilled'],
            f'timeEnemySpentControlled_{i}': players[str(i)]['timeEnemySpentControlled'],
            f'totalGold_{i}': players[str(i)]['totalGold'],
            f'xp_{i}': players[str(i)]['xp']
        }

        #Now that I have the current players stats, extend it to the overall players_dict
        players_dict.update(temp_dict)
        
    #Update the players_dict one more time with the timestamp for the timeframe
    players_dict.update({'timestamp' : timeframe['timestamp']})
    
    #Append the players_dict to the overall player_stats list of dicts
    player_stats.append(players_dict)

In [12]:
#Now convert player_stats to a df
player_stats = pd.DataFrame(player_stats)

In [13]:
player_stats.head()

,currentGold_1,magicDamageDoneToChampions_1,physicalDamageDoneToChampions_1,trueDamageDoneToChampions_1,totalDamageDoneToChampions_1,goldPerSecond_1,jungleMinionsKilled_1,level_1,minionsKilled_1,timeEnemySpentControlled_1,totalGold_1,xp_1,currentGold_2,magicDamageDoneToChampions_2,physicalDamageDoneToChampions_2,trueDamageDoneToChampions_2,totalDamageDoneToChampions_2,goldPerSecond_2,jungleMinionsKilled_2,level_2,minionsKilled_2,timeEnemySpentControlled_2,totalGold_2,xp_2,currentGold_3,magicDamageDoneToChampions_3,physicalDamageDoneToChampions_3,trueDamageDoneToChampions_3,totalDamageDoneToChampions_3,goldPerSecond_3,jungleMinionsKilled_3,level_3,minionsKilled_3,timeEnemySpentControlled_3,totalGold_3,xp_3,currentGold_4,magicDamageDoneToChampions_4,physicalDamageDoneToChampions_4,trueDamageDoneToChampions_4,totalDamageDoneToChampions_4,goldPerSecond_4,jungleMinionsKilled_4,level_4,minionsKilled_4,timeEnemySpentControlled_4,totalGold_4,xp_4,currentGold_5,magicDamageDoneToChampions_5,physicalDamageDoneToChampions_5,trueDamageDoneToChampions_5,totalDamageDoneToChampions_5,goldPerSecond_5,jungleMinionsKilled_5,level_5,minionsKilled_5,timeEnemySpentControlled_5,totalGold_5,xp_5,currentGold_6,magicDamageDoneToChampions_6,physicalDamageDoneToChampions_6,trueDamageDoneToChampions_6,totalDamageDoneToChampions_6,goldPerSecond_6,jungleMinionsKilled_6,level_6,minionsKilled_6,timeEnemySpentControlled_6,totalGold_6,xp_6,currentGold_7,magicDamageDoneToChampions_7,physicalDamageDoneToChampions_7,trueDamageDoneToChampions_7,totalDamageDoneToChampions_7,goldPerSecond_7,jungleMinionsKilled_7,level_7,minionsKilled_7,timeEnemySpentControlled_7,totalGold_7,xp_7,currentGold_8,magicDamageDoneToChampions_8,physicalDamageDoneToChampions_8,trueDamageDoneToChampions_8,totalDamageDoneToChampions_8,goldPerSecond_8,jungleMinionsKilled_8,level_8,minionsKilled_8,timeEnemySpentControlled_8,totalGold_8,xp_8,currentGold_9,magicDamageDoneToChampions_9,physicalDamageDoneToChampions_9,trueDamageDoneToChampions_9,totalDamageDoneToChampions_9,goldPerSecond_9,jungleMinionsKilled_9,level_9,minionsKilled_9,timeEnemySpentControlled_9,totalGold_9,xp_9,currentGold_10,magicDamageDoneToChampions_10,physicalDamageDoneToChampions_10,trueDamageDoneToChampions_10,totalDamageDoneToChampions_10,goldPerSecond_10,jungleMinionsKilled_10,level_10,minionsKilled_10,timeEnemySpentControlled_10,totalGold_10,xp_10,timestamp
0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,500,0,0,0,0,0,0,1,0,0,500,0,0
1,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,20,0,1,0,0,500,0,50,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,0,0,1,0,0,500,0,0,0,0,0,0,20,0,1,0,0,500,0,60020
2,63,0,0,0,0,0,0,1,2,0,563,181,186,0,253,0,253,0,6,2,0,10836,686,449,112,0,78,0,78,0,0,1,5,3566,612,240,63,0,0,34,34,0,0,1,2,3933,563,75,44,0,138,0,138,20,0,1,1,4887,544,135,71,95,76,0,172,0,0,1,0,0,521,0,121,0,0,0,0,0,4,2,0,0,621,320,126,189,0,0,189,0,0,1,6,10470,626,270,98,0,0,0,0,0,0,1,4,0,598,199,43,73,0,0,73,20,0,1,0,0,543,18,120051
3,318,0,107,0,107,0,0,3,10,0,818,725,554,0,253,0,253,0,16,3,0,49180,1054,833,879,67,562,54,683,0,0,3,16,8258,1389,1018,632,0,438,110,549,0,0,2,10,20348,1132,393,336,0,619,90,709,20,0,2,3,8854,836,393,421,239,76,0,315,0,0,2,11,0,871,634,433,0,0,0,0,0,12,3,0,3897,933,685,395,209,39,0,248,0,0,3,14,38774,895,662,297,0,824,0,824,0,0,2,8,0,797,428,277,265,241,70,577,20,0,1,0,4459,777,187,180064
4,647,336,240,0,577,0,0,3,13,1486,1147,1109,1192,31,643,0,674,0,20,3,0,74848,1692,1085,688,172,1351,54,1577,0,0,4,17,12717,1948,1222,165,0,678,155,834,0,0,3,22,37257,1765,938,654,0,771,102,873,20,0,3,5,17377,1154,938,621,535,76,0,611,0,0,3,16,0,1071,994,876,0,676,88,764,0,12,3,0,28375,1376,823,783,356,84,0,44

Now, that I have all of the player stats from one match in this dataframe, I can simply access the index corresponding to the timeframe I want. Normally for this project, we would access the timeframe corresponding to the 20 minute mark, however this particular match didn't last that long.

In [14]:
len(player_stats)

19

In [15]:
player_stats.tail()

,currentGold_1,magicDamageDoneToChampions_1,physicalDamageDoneToChampions_1,trueDamageDoneToChampions_1,totalDamageDoneToChampions_1,goldPerSecond_1,jungleMinionsKilled_1,level_1,minionsKilled_1,timeEnemySpentControlled_1,totalGold_1,xp_1,currentGold_2,magicDamageDoneToChampions_2,physicalDamageDoneToChampions_2,trueDamageDoneToChampions_2,totalDamageDoneToChampions_2,goldPerSecond_2,jungleMinionsKilled_2,level_2,minionsKilled_2,timeEnemySpentControlled_2,totalGold_2,xp_2,currentGold_3,magicDamageDoneToChampions_3,physicalDamageDoneToChampions_3,trueDamageDoneToChampions_3,totalDamageDoneToChampions_3,goldPerSecond_3,jungleMinionsKilled_3,level_3,minionsKilled_3,timeEnemySpentControlled_3,totalGold_3,xp_3,currentGold_4,magicDamageDoneToChampions_4,physicalDamageDoneToChampions_4,trueDamageDoneToChampions_4,totalDamageDoneToChampions_4,goldPerSecond_4,jungleMinionsKilled_4,level_4,minionsKilled_4,timeEnemySpentControlled_4,totalGold_4,xp_4,currentGold_5,magicDamageDoneToChampions_5,physicalDamageDoneToChampions_5,trueDamageDoneToChampions_5,totalDamageDoneToChampions_5,goldPerSecond_5,jungleMinionsKilled_5,level_5,minionsKilled_5,timeEnemySpentControlled_5,totalGold_5,xp_5,currentGold_6,magicDamageDoneToChampions_6,physicalDamageDoneToChampions_6,trueDamageDoneToChampions_6,totalDamageDoneToChampions_6,goldPerSecond_6,jungleMinionsKilled_6,level_6,minionsKilled_6,timeEnemySpentControlled_6,totalGold_6,xp_6,currentGold_7,magicDamageDoneToChampions_7,physicalDamageDoneToChampions_7,trueDamageDoneToChampions_7,totalDamageDoneToChampions_7,goldPerSecond_7,jungleMinionsKilled_7,level_7,minionsKilled_7,timeEnemySpentControlled_7,totalGold_7,xp_7,currentGold_8,magicDamageDoneToChampions_8,physicalDamageDoneToChampions_8,trueDamageDoneToChampions_8,totalDamageDoneToChampions_8,goldPerSecond_8,jungleMinionsKilled_8,level_8,minionsKilled_8,timeEnemySpentControlled_8,totalGold_8,xp_8,currentGold_9,magicDamageDoneToChampions_9,physicalDamageDoneToChampions_9,trueDamageDoneToChampions_9,totalDamageDoneToChampions_9,goldPerSecond_9,jungleMinionsKilled_9,level_9,minionsKilled_9,timeEnemySpentControlled_9,totalGold_9,xp_9,currentGold_10,magicDamageDoneToChampions_10,physicalDamageDoneToChampions_10,trueDamageDoneToChampions_10,totalDamageDoneToChampions_10,goldPerSecond_10,jungleMinionsKilled_10,level_10,minionsKilled_10,timeEnemySpentControlled_10,totalGold_10,xp_10,timestamp
14,316,5347,1640,459,7448,0,0,10,73,43147,5776,6518,2318,146,2515,0,2661,0,87,9,6,260446,5893,6013,1968,426,3487,408,4321,0,1,10,95,51243,6578,6872,216,0,4054,419,4474,0,0,8,100,77230,5791,4725,260,0,2648,454,3103,30,0,7,23,51526,4160,4017,301,5630,168,366,6165,0,0,10,81,36466,5251,6496,289,0,3099,88,3187,0,48,8,11,74446,4539,4319,876,1416,127,0,1544,0,0,9,88,262222,3931,5446,1188,258,3383,0,3641,0,0,8,95,14801,4313,4167,651,1819,763,354,2937,30,0,7,2,18860,2926,3383,840308
15,271,5347,1640,459,7448,0,0,10,75,43147,6096,6919,2639,205,2720,0,2925,0,95,10,6,260512,6214,6521,630,510,4419,408,5338,0,1,10,95,52003,7140,7204,1401,0,6577,1020,7598,0,0,9,107,78650,6976,5367,1297,0,2794,644,3439,30,0,8,26,54036,5197,4816,696,5630,168,366,6165,0,0,10,93,41452,5646,7160,581,0,3564,164,3728,0,52,8,11,79103,4831,4824,1198,2210,127,0,2338,0,0,9,88,269190,4253,5946,310,285,5058,0,5344,0,0,8,95,18499,5035,4776,427,2166,917,392,3476,30,0,7,2,20710,3177,3603,900329
16,408,6456,1805,488,8750,0,0,10,76,44535,6233,7039,93,205,2720,0,2925,0,103,10,6,271114,6568,6992,997,568,5417,408,6394,0,7,11,101,57319,7507,7650,724,0,8206,1413,9619,0,0,9,107,81490,7399,5662,1479,0,2794,644,3439,30,0,9,28,56281,5379,5293,282,7364,168,366,7899,0,0,11,96,53379,6132,8021,799,0,3564,240,3804,0,56,9,11,94592,5049,5112,1657,2530,127,0,2658,0,0,10,104,290925,4712,6823,663,368,5876,101,6346,0,0,9,112,30523,5888,5646,797,2777,965,392,4135,30,0,8,2,25103,3547,4363,960345
17,658,6456,1805,488,8750,0,4,11,78,48633,6483,7446,585,205,2720,0,2925,0,107,11,16,306657,7060,8024,1148,659,6823,570,8052,0,7,11,103,61778,7658,79

In [16]:
player_stats.loc[18]

currentGold_1                         1505
magicDamageDoneToChampions_1          6877
physicalDamageDoneToChampions_1       1805
trueDamageDoneToChampions_1            519
totalDamageDoneToChampions_1          9202
                                    ...   
minionsKilled_10                         3
timeEnemySpentControlled_10          29330
totalGold_10                          4570
xp_10                                 4941
timestamp                          1066945
Name: 18, Length: 121, dtype: int64

# Gather Match IDs

Write a function to gather match IDs using puuids.

In [17]:
from time import sleep

#Create the function
def get_match_ids(puuids, api_key):
    """
    This function takes in a list of puuids and a riot games api key to gather
    and return a list of match ids.
    """
    
    #Create an empty list to store the match ids
    matches = []
    
    #Set the base url
    url = 'https://americas.api.riotgames.com/'
    
    #Set a counter var
    n = 0
    
    #Create another var to store the length of puuids
    num_puuids = len(puuids)
    
    #Loop through the puuids and gather 100 match ids from each of them
    for puuid in puuids:
        #Update counter
        n += 1

        #Build the query
        query = f'lol/match/v5/matches/by-puuid/{puuid}/ids?start={0}&count={100}&api_key={api_key}'

        #Get the response
        response = requests.get(url + query)

        #Check response and leave a status message
        if response.status_code != 200:
            print(f'Something went wrong! Status Code {response.status_code} for puuid {puuid}. Skipping this puuid.\n')
            continue

        #Unpack the response as json
        #This should be a list of match ids
        data = response.json()

        #Extend the matches list with the data
        matches.extend(data)
        
        #Leave a status message after every 2 iterations (~6 minutes)
        if n % 2 == 0:
            print(f'Completed {n} of {num_puuids} puuids.')
            print(f'Total match IDs gathered: {len(matches)}\n')
        
        sleep(180)
        
    return matches

# Get My Own PUUIDs

In [25]:
def get_puuid(name_list,df,api_key):
    counter = 0
    for name in name_list:
        counter += 1
        if counter%50 == 0:
            print(counter)
            print('function paused')
            print
            sleep(180)
            print('Continue')
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            if len(summoner_info) == 7:
                info = {'puuid':summoner_info['puuid'],
                       'username':summoner_info['name']}
                df = df.append(info,ignore_index = True)
            else:
                print(summoner_info)
        else:
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            if len(summoner_info) == 7:
                info = {'puuid':summoner_info['puuid'],
                       'username':summoner_info['name']}
                df = df.append(info,ignore_index = True)
            else:
                print(summoner_info)
    return df

In [19]:
def automated_puuid(name_list,df,api_key):
    df = get_puuid(name_list,username_df,api_key)
    puuid_list = df.puuid
    matches = get_match_ids(puuid_list, api_key)
    return matches

In [20]:
#IMPORT 2640 players.csv
df = pd.read_csv('2640_players.csv')

In [21]:
# make the players name column into a series
player_list = df.players

In [22]:
# I will gather the values for 1200 - 1650
new_player_list = player_list[1200:1650]

In [23]:
# make an empty dataframe
username_df = pd.DataFrame(columns = ['puuid','username'])

In [26]:
# Get the match IDs using the automated_puuid function
match_list = automated_puuid(new_player_list,username_df,api_key)

50
function paused
Continue
100
function paused
Continue
{'status': {'message': 'Data not found - summoner not found', 'status_code': 404}}
150
function paused
Continue
200
function paused
Continue
250
function paused
Continue
300
function paused
Continue
350
function paused
Continue
400
function paused
Continue
450
function paused
Continue
Completed 2 of 449 puuids.
Total match IDs gathered: 200

Completed 4 of 449 puuids.
Total match IDs gathered: 400

Completed 6 of 449 puuids.
Total match IDs gathered: 600

Completed 8 of 449 puuids.
Total match IDs gathered: 800

Completed 10 of 449 puuids.
Total match IDs gathered: 1000

Completed 12 of 449 puuids.
Total match IDs gathered: 1200

Completed 14 of 449 puuids.
Total match IDs gathered: 1400

Completed 16 of 449 puuids.
Total match IDs gathered: 1600

Completed 18 of 449 puuids.
Total match IDs gathered: 1800

Completed 20 of 449 puuids.
Total match IDs gathered: 2000

Completed 22 of 449 puuids.
Total match IDs gathered: 2200

Compl

Something went wrong! Status Code 403 for puuid at67mjuY-VJ88WVSu8wcO7sQqLU660IvlbS01B4WETEmSso-e6iR4PIG5Odo_znmnFg9s7sxoX25hA. Skipping this puuid.

Something went wrong! Status Code 403 for puuid pVG693UrhQou5N9ZXdXV64q-kmthMkgf5LT8l441ZfB19BAcLn5QJqIx13bl2N_5_Cg_lwdHflvayA. Skipping this puuid.

Something went wrong! Status Code 403 for puuid TsOiBWpjMaht4dKLZcMeZftxjWLHsBWQllOyIVDKjZlqB-yA6RL02aTOaFfjadwSEfKDFi89qaE95w. Skipping this puuid.

Something went wrong! Status Code 403 for puuid hPtZsbywWBU-6zkJDhld2puU-3PQEWOxD-HjbEiuDilTzPstaM_5VTAK8CUJIpMZi2R9c2YNySdoiA. Skipping this puuid.

Something went wrong! Status Code 403 for puuid rNdANLJsu2Pb26UJcS1ZG7FFNWlBuD3HbF74-KkhDHsu_GiMB7WEMZInEDI5wtjeoBWsAqhmrb9PFw. Skipping this puuid.

Something went wrong! Status Code 403 for puuid O6tV8IC1hpkVxEVK-ds8eXeCLmZQA2o29w1NykKjkHsTBwMeQu04vDV-W1XkiPZ88qhqkIYCsDSkzg. Skipping this puuid.

Something went wrong! Status Code 403 for puuid dDhWXgGRaCJ7LQVBRqESXFfjgTZTKcOaHwAI2DJqxer7N8zk5Vxy

Something went wrong! Status Code 403 for puuid hlE0j_ktrHyR1gJKXVDhetFefVs7yGepktK6n4GtlDckwAhSk7FXWNajSXG6PUWGInhxTDJl0y8aVQ. Skipping this puuid.

Something went wrong! Status Code 403 for puuid 2JBygqMDQMyskCnC_BXKZKiUEa5hD9FpsG3hhYuXH963o6kbPpsv5aFVZlR5-mUbbIE3uwL4J5KBpg. Skipping this puuid.

Something went wrong! Status Code 403 for puuid w-xa5fjp8fczK3NWAhvLCKmUOHqc21ErA4AHDPRFvrp5DcCWYmnR52vpkJzJvfpnfiL79U4t5E-BTA. Skipping this puuid.

Something went wrong! Status Code 403 for puuid Oc7PC1_KSUTh0zgU9mDdxT_wbCPbus-cIlsOYPjB_YAEpkJJoI4S98WD3zdI-3uYrBoRpCIOCK_Aag. Skipping this puuid.

Something went wrong! Status Code 403 for puuid hpg4tru7UwQlNaiaV0FMujO6QfvOJA3PnoGM6K8Qxrgf2R92h-oVxF9-ATvPLhcY_5gzTjVsGnN4Qg. Skipping this puuid.

Something went wrong! Status Code 403 for puuid zZ9gOCXJ3iQEB8Klx2Cs8xIkZbJoTkJV_R1SH_AKlyB3yoctNWayajLmWer_Uznxt7NHvD0SiU-N-g. Skipping this puuid.

Something went wrong! Status Code 403 for puuid NRwM0g2f5vp__7vFJATXFwSlJ6InEMwKfrgk8W5vzsfQ68PhVg6J

Something went wrong! Status Code 403 for puuid i4KKDZNVCm2iUaZYGf05JG4FhF2K5p-Hf41krHlGFu3BqCcU55I7pZXLBa1P_6KDvSA7vcxUJXmQHw. Skipping this puuid.

Something went wrong! Status Code 403 for puuid xGd7R9pCj3dzVz0nInH_Qui5lsaolR5kZpjqWnIwAiPhBJqz51b-pJxoQXcSob4FElPvwP79UNuOpg. Skipping this puuid.

Something went wrong! Status Code 403 for puuid e6gBwnV5l6eXmareM29alzc6iQb4u_C8LBor_E6mq2EFlHTxE1YTrT35ndy98keavvGIdzt6bF3AnQ. Skipping this puuid.

Something went wrong! Status Code 403 for puuid I850JrPgQcbWQxPu8LMitWIpG1HkH9mvsXSzuuV5cuSadYTAhbI6bo_XCEE8IAInZUuo-p34kPbSUw. Skipping this puuid.

Something went wrong! Status Code 403 for puuid qBoCkE5DPV0a6Na5lWTwhjaFeeb5rRcExL6st1xqmQbArwvwiu63sm9xWheCLGhLJnmnFRP6J5MFWg. Skipping this puuid.

Something went wrong! Status Code 403 for puuid 8bF9sq6hyryPL6VI2xQ_U-SgG_plt6qxjvML8X4FiSEnedtT14rF4Dcs4xrMyvNOQdAfpJqlikovBw. Skipping this puuid.

Something went wrong! Status Code 403 for puuid TFiPfmzCfAR4sJLFICTKO6HOwbJNdoFJt4SjzDXX1gtLgbCH99jg

Something went wrong! Status Code 403 for puuid nv44jpDd__Dnkb6U2Eoj85_P-GfRIJmfzL8-sd2L2bRqZzjDsGGb-HSGB0hDYlRPLeHQBRkA3OMuHQ. Skipping this puuid.

Something went wrong! Status Code 403 for puuid N1XFDCl47154L_0EL07zY6M9GFEdq9NoUdxD3vvtHnqYyvQhcjFRNdWYHyxYMXBGzgDeHh1wYprFtg. Skipping this puuid.

Something went wrong! Status Code 403 for puuid JKVH6LSgGCKwnowMT4cd-a1a1h2GHhy9cqdATq9XdFkIJ0McGgDVgRfE3RgcT8i52kaFh4wKdJ6tsQ. Skipping this puuid.

Something went wrong! Status Code 403 for puuid l4bWCliVmd3YxpAJDqxdCoiqHlrDzjSfdi0DFNT60XSuKlN44S9FClBmoVJZ9m-yy7JPeqRcWuCQyw. Skipping this puuid.

Something went wrong! Status Code 403 for puuid U5YVtJ19yBM5CcTvteBB6vDwasgibIPgFaJo3wkrsN3dZ1GWZt7U_YkcH37yh4jz89SVghjYLS4Maw. Skipping this puuid.

Something went wrong! Status Code 403 for puuid Zesk23i0jT7rEPMHpSvoklvOp8i9huaF1PvT8mZ3DYggwJyxntjVy3asljBiGAQSgiGcchQA7RdGfw. Skipping this puuid.

Something went wrong! Status Code 403 for puuid 3FFA5rD1ANXwnuf3jt9FG3ZB0OPG0l8oPqbGgqFvY6Lj5-8B2KSO

In [27]:
len(match_list)

22582

In [31]:
#Convert the list to a set to get rid of duplicates
match_list = set(match_list)

In [32]:
len(match_list)

20318

In [39]:
import json

#Convert it to a df
matches_df = pd.DataFrame(match_list)

In [40]:
matches_df

,0
0,NA1_4062082172
1,NA1_4093908534
2,NA1_4095439496
3,NA1_4036127439
4,NA1_4100254813
...,...
20313,NA1_4091434839
20314,NA1_4105324956
20315,NA1_4088568563
20316,NA1_4091064163


In [43]:
#Save as csv file
matches_df.to_csv('match_ids_smith.csv', index = False)

In [44]:
#Load the file for testing
matches_df = pd.read_csv('match_ids_smith.csv')
matches_df

,0
0,NA1_4062082172
1,NA1_4093908534
2,NA1_4094915650
3,NA1_4082128299
4,NA1_4083500908
...,...
20313,NA1_4088936717
20314,NA1_4049964605
20315,NA1_4079861676
20316,NA1_4064742197


# Joining Match ID Files

Figure out how to join multiple csv files so that we have a complete list of match_ids.

In [ ]:
chris_ids = pd.read_csv('')